# What's Inside a Data Query Engine  
## *Building one from Scratch*  

## Part 2: Just A Tad More Detail 
  
![What's Inside a Data Query Engine](./images/dataengine05.png)

### <font color='green'>__Support for Google Colab__  </font>  
    
open this notebook in Colab using the following button:  
  
<a href="https://colab.research.google.com/github/shauryashaurya/learn-data-munging/blob/main/00-Python-Collections/01.03%20Fun%20with%20Functools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>  

  
<font color='green'>uncomment and execute the cell below to setup and run this notebook on Google Colab.</font>

In [104]:
# # SETUP FOR COLAB: select all the lines below and uncomment (CTRL+/ on windows)
# # Let's download and unzip the Small MovieLens Dataset
# ! mkdir ./../data
# ! wget -q https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# ! unzip ./ml-latest-small.zip -d ./../data/

### Get the _Small_ MovieLens Dataset

We'll use the [small MovieLens dataset](https://grouplens.org/datasets/movielens/#:~:text=Small%3A%20100%2C000%20ratings%20and%203%2C600%20tag%20applications) here.

Download it and unzip to the data folder under the name `ml-latest-small`.

This dataset expands to about 3.2 MB on your local disk. 

In [105]:
datalocation = "./data/ml-latest-small/"

In [106]:
# specify file names
file_path_movies = datalocation + "movies.csv"
file_path_links = datalocation + "links.csv"
file_path_ratings = datalocation + "ratings.csv"
file_path_tags = datalocation + "tags.csv"

# Here's what our data engine should be able to do  
* Load the data into the memory and capture some metadata (things like column names, data types etc.)  
* Get a query, a SELECT (xxx) FROM (xxx) WHERE (XXX)  
* Parse the query to make sense of it  
* Highlight if there are any errors  
* Build a query plan  
* By looking at the plan and metadata, optimize the query futher  
* Execute the query  
* Show the results  
* Show the cost of running the query  
  
  
_The full set of notebooks also covers JOINs and nested queries, but we are going to treat them as intermediate to advanced cases - since they may distract us from the goal of just being able to understand how data engines work._

We'll directly use the [CSV module](https://docs.python.org/3/library/csv.html) here just to keep our focus on the data engine itself and not get distracted by the intricacies of loading a CSV file.

# SQL Parser  
  
converts SQL queries into a structured Abstract Syntax Tree (AST). 
AST is a tree representation of the syntactic structure of the SQL code.  
  

In [5]:
import csv
import re
from typing import List, NamedTuple
from enum import Enum, auto

## Tokenize

In [6]:
class TokenType(Enum):
	SELECT = auto()
	ASTERISK = auto()
	FROM = auto()
	WHERE = auto()
	JOIN = auto()
	ON = auto()
	ORDER = auto()
	BY = auto()
	GROUP = auto()
	HAVING = auto()
	INSERT = auto()
	UPDATE = auto()
	DELETE = auto()
	IDENTIFIER = auto()
	STRING = auto()
	NUMBER = auto()
	OPERATOR = auto()
	PUNCTUATION = auto() # ignoring punctuation could be problematic, it's better to support proper COMMA, SEMICOLON etc.
	WHITESPACE = auto()  # so we can ignore it in further processing

In [7]:
# Define a dictionary for quick keyword lookup
SQL_KEYWORDS = {
	'SELECT': TokenType.SELECT,
	'FROM': TokenType.FROM,
	'WHERE': TokenType.WHERE,
	'JOIN': TokenType.JOIN,
	'ON': TokenType.ON,
	'ORDER': TokenType.ORDER,
	'BY': TokenType.BY,
	'GROUP': TokenType.GROUP,
	'HAVING': TokenType.HAVING,
	'INSERT': TokenType.INSERT,
	'UPDATE': TokenType.UPDATE,
	'DELETE': TokenType.DELETE,
	'ASTERISK': TokenType.ASTERISK
}

In [8]:
def tokenize(sql):
	token_patterns = r'''
		('[^']*'|"[^"]*")			  # String literals
	  | (<=|>=|<>|!=|<|>|=)			  # Comparison operators
	  | (\d+\.\d*|\.\d+|\d+)		  # Numeric values
	  | ([,;()])					  # Punctuation
	  | (\b[a-zA-Z_][a-zA-Z0-9_]*\b)  # Identifiers or SQL keywords
	  | (\s+)						  # Whitespace
	'''
	token_regex = re.compile(token_patterns, re.VERBOSE) #VERBOSE allows for multiline regex with comments 
	for match in token_regex.finditer(sql):
		token = match.group(0)
		if token.isspace():
			yield (token, TokenType.WHITESPACE)
		elif token in (',', ';', '(', ')'):
			yield (token, TokenType.PUNCTUATION)
		elif token in ('*'):
			yield (token, TokenType.ASTERISK)
		elif token.upper() in SQL_KEYWORDS:
			yield (token.upper(), SQL_KEYWORDS[token.upper()])
		elif re.match(r'^[\'"].*[\'"]$', token):
			yield (token, TokenType.STRING)
		elif re.match(r'^\d+(\.\d+)?$', token):
			yield (token, TokenType.NUMBER)
		elif re.match(r'<=|>=|<>|!=|<|>|=$', token):
			yield (token, TokenType.OPERATOR)
		else:
			yield (token, TokenType.IDENTIFIER)

In [9]:
# Test
sql_query_test_01 = "SELECT name, age FROM users WHERE age >= 21 AND status = 'active' ORDER BY age DESC;"
t = tokenize(sql_query_test_01)
print(list(t))
# print(next(t,None))

[('SELECT', <TokenType.SELECT: 1>), (' ', <TokenType.WHITESPACE: 19>), ('name', <TokenType.IDENTIFIER: 14>), (',', <TokenType.PUNCTUATION: 18>), (' ', <TokenType.WHITESPACE: 19>), ('age', <TokenType.IDENTIFIER: 14>), (' ', <TokenType.WHITESPACE: 19>), ('FROM', <TokenType.FROM: 3>), (' ', <TokenType.WHITESPACE: 19>), ('users', <TokenType.IDENTIFIER: 14>), (' ', <TokenType.WHITESPACE: 19>), ('WHERE', <TokenType.WHERE: 4>), (' ', <TokenType.WHITESPACE: 19>), ('age', <TokenType.IDENTIFIER: 14>), (' ', <TokenType.WHITESPACE: 19>), ('>=', <TokenType.OPERATOR: 17>), (' ', <TokenType.WHITESPACE: 19>), ('21', <TokenType.NUMBER: 16>), (' ', <TokenType.WHITESPACE: 19>), ('AND', <TokenType.IDENTIFIER: 14>), (' ', <TokenType.WHITESPACE: 19>), ('status', <TokenType.IDENTIFIER: 14>), (' ', <TokenType.WHITESPACE: 19>), ('=', <TokenType.OPERATOR: 17>), (' ', <TokenType.WHITESPACE: 19>), ("'active'", <TokenType.STRING: 15>), (' ', <TokenType.WHITESPACE: 19>), ('ORDER', <TokenType.ORDER: 7>), (' ', <Toke

## AST Node Definitions

In [10]:
class ASTNode:
    pass

In [87]:
class SelectStatement(ASTNode):
	def __init__(self, columns, table_name, where_clause=None):
		self.columns = columns  # List of column names or '*'
		self.table_name = table_name  # Name of the table
		self.where_clause = where_clause  # WhereClause node or None
		print("SelectStatement: this Select Statement has \nColumns: ",str(list(self.columns)),"\nTable: ",str(self.table_name),"\nWHERE CLAUSE: ", str(self.where_clause))
		
	# def __repr__(self):
		# return "Select Statement has \nColumns: "+str(list(self.columns)),"\nTable: "+str(self.table_name)+"and WHERE CLAUSE: "+ str(self.where_clause)

In [88]:
class WhereClause(ASTNode):
    def __init__(self, condition):
        self.condition = condition  # This could be a more complex structure in a full implementation

## Parser

In [89]:
# wish this was smaller
class Parser:
	def __init__(self, tokens):
		self.tokens = tokens
		self.current_token = None
		self.next_token = None
		self._next_token()

	def _next_token(self):
		"""Advance to the next token."""
		try:
			# print("Parse: _next_token: self.current_token = ",self.current_token)
			# print("Parse: _next_token: self.next_token = ",self.next_token)
			
			self.current_token = self.next_token
			self.next_token = next(self.tokens, None)
		except StopIteration:
			self.current_token = None

	def parse(self):
		"""Parse the tokens into an AST."""
		if (self.current_token == None) and (self.next_token != None):
			self._next_token()
		# 
		if self.current_token[1] != TokenType.SELECT:
			raise SyntaxError("Query must start with SELECT")
		self._next_token()

		columns = self._parse_columns()

		print("Parse: parse: columns = ",columns)

		# skip whitespace
		if self.current_token[1] == TokenType.WHITESPACE:
			while self.current_token[1] == TokenType.WHITESPACE:
				self._next_token()

		if self.current_token[1] != TokenType.FROM:
			raise SyntaxError("Expected FROM after column list")
		self._next_token()

		# print("Parse:parse: Now parsing table names: self.current_token = ",self.current_token) 

		table_name = self._parse_table_name()

		print("Parse: parse: table_name = ",table_name)

		where_clause = None
		if self.current_token[1] == TokenType.WHERE:
			self._next_token()
			where_clause = self._parse_where_clause()

		print("Parse:parse: where_clause = ", where_clause)

		return SelectStatement(columns, table_name, where_clause)

	def _parse_columns(self):
		"""Parse the columns part of the SELECT statement."""
		columns = []

		# skip whitespace
		while self.current_token[1] == TokenType.WHITESPACE:
			# print("Parse: _parse_columns: ignoring whitespace")
			# print("Parse: _parse_columns: self.current_token",self.current_token)
			self._next_token()
			# print("Parse: _parse_columns: NEW self.current_token",self.current_token)
		
		# print("Parser: _parse_columns: self.current_token = ",self.current_token[0],self.current_token[1])
		if self.current_token[1] == TokenType.ASTERISK:
			columns.append('*')
			self._next_token()
		else:
			while True:
				# print("Parse: _parse_columns: in WHILE - self.current_token",self.current_token)
				# skip whitespace or commas
				while self.current_token[1] == TokenType.WHITESPACE or self.current_token[1] == TokenType.PUNCTUATION:
					# print("Parse: _parse_columns: in WHILE2 - ignoring whitespace")
					# print("Parse: _parse_columns: in WHILE2 - self.current_token",self.current_token)
					self._next_token()
					# print("Parse: _parse_columns: in WHILE2 - ignoring whitespace - NEW self.current_token",self.current_token)
				
				# do this till you reach from, we do not support sub-queries in this engine yet.
				if self.current_token[1] == TokenType.FROM:
					# print("Parse: _parse_columns: found FROM - self.current_token = ",self.current_token)
					break
					
				if self.current_token[1] != TokenType.IDENTIFIER:
					raise SyntaxError("Expected column name")
				print("add ",self.current_token[0]," to list of columns")
				columns.append(self.current_token[0])
				self._next_token()
				# print("Parse: _parse_columns: self.current_token = ",self.current_token)

				# if self.current_token[1] != TokenType.COMMA:
				if self.current_token[1] == TokenType.PUNCTUATION:
					# print("Parse: _parse_columns: ignoring TokenType.PUNCTUATION")
					# print("Parse: _parse_columns: skip punctuation - self.current_token = ",self.current_token)
					continue

					
				# self._next_token()  # Skip the comma or the whitespace

		# print("Parse: _parse_columns: self.current_token",self.current_token)
		# print("Parse: _parse_columns: columns = ",columns)
		
		return columns

	def _parse_table_name(self):
		"""Parse the table name."""
		# skip whitespace
		while self.current_token[1] == TokenType.WHITESPACE:
			# print("Parse: _parse_table_name: ignoring whitespace")
			# print("Parse: _parse_table_name: self.current_token",self.current_token)
			self._next_token()
			# print("Parse: _parse_table_name: NEW self.current_token",self.current_token)
		if self.current_token[1] != TokenType.IDENTIFIER:
			raise SyntaxError("Expected table name")
		table_name = self.current_token[0]
		self._next_token()
		return table_name

	# TODO
	def _parse_where_clause(self):
		"""Parse the WHERE clause."""
		# In a full implementation, this would need to handle complex expressions.
		# For simplicity, we'll assume it's just a single condition.
		if self.current_token[1] != TokenType.IDENTIFIER and self.current_token[1] != TokenType.WHITESPACE:
			raise SyntaxError("Expected condition after WHERE")
		# condition = self.current_token[0]
		condition = None
		# self._next_token()
		return WhereClause(condition)



In [90]:
# Example Usage
sql_query_test_02 = "SELECT name, age FROM users WHERE age > 30"
print("SQL Query:\n\t", sql_query_test_02)
tokens = tokenize(sql_query_test_02) # separate, as you list(gen) a generator, it's already emitted all values...
# print("Tokenized:\n\t",list(tokens_check), " ", tokens_check)
parser = Parser(tokens)
# print("Parser:\n\t",tokens)
ast = parser.parse()

SQL Query:
	 SELECT name, age FROM users WHERE age > 30
add  name  to list of columns
add  age  to list of columns
Parse: parse: columns =  ['name', 'age']
Parse: parse: table_name =  users
Parse:parse: where_clause =  None
SelectStatement: this Select Statement has 
Columns:  ['name', 'age'] 
Table:  users 
WHERE CLAUSE:  None


In [91]:
# Printing the AST for demonstration
print(ast)

# Storage Layer

In [213]:
class Column:
    def __init__(self, name, col_type):
        self.name = name
        self.col_type = col_type

In [214]:
class Table:
	def __init__(self, name, columns):
		self.name = name
		self.columns = {col.name: col for col in columns}
		self.data = []

	def table_scan(self):
		return self.data

	def insert(self, row):
		if set(row.keys()) != set(self.columns.keys()):
			raise ValueError("Row does not match table schema")
		# Type checking and conversion can be added here
		self.data.append(row)

	def select(self, columns, limit = -1, where_clause=None):
		result = []
		counter = 0
		for row in self.data:
			counter += 1
			if (counter<=limit or limit == -1) and (where_clause is None or where_clause.evaluate(row)):
				result_row = {col: row[col] for col in columns}
				result.append(result_row)
			else:
				break
		return result

	# Todo: Additional methods for update, delete etc.

In [215]:
def convert_type(value, col_type):
    """
    Convert the string value from CSV to the specified data type.
    """
    if col_type == int:
        return int(value)
    elif col_type == float:
        return float(value)
    elif col_type == str:
        return value
    else:
        raise TypeError(f"Unsupported column type: {col_type}")

In [216]:
class Database:
	def __init__(self):
		self.tables = {}

	def create_table(self, name, columns):
		self.tables[name] = Table(name, columns)

	def get_table(self, name):
		return self.tables.get(name)
	
	def load_csv(self, table_name, file_path, delimiter=',', quotechar='"', escapechar=None, quoting=csv.QUOTE_MINIMAL):
		"""
		Load data from a CSV file into the specified table with support for different CSV dialects.
		"""
		print("Database: load_csv: file_path = ",file_path)
		table = self.get_table(table_name)
		if not table:
			raise ValueError(f"Table {table_name} does not exist")

		with open(file_path, 'r', encoding='utf-8') as file:
			reader = csv.DictReader(file, delimiter=delimiter, quotechar=quotechar, escapechar=escapechar, quoting=quoting)
			for row in reader:
				converted_row = {col: convert_type(row[col], table.columns[col].col_type) for col in row}
				table.insert(converted_row)

In [217]:
# Test this
db = Database()
db.create_table('users', [Column('name', str), Column('age', int)])
db.get_table('users').insert({'name': 'Alice', 'age': 30})
print(db.get_table('users').select(['name', 'age']))

[{'name': 'Alice', 'age': 30}]


In [218]:
# Example Usage

# movieId,title,genres

db = Database()
db.create_table('movies', [Column('movieId', str), Column('title', str), Column('genres', str)])
db.load_csv('movies', file_path_movies)

Database: load_csv: file_path =  ./data/ml-latest-small/movies.csv


In [219]:
# lists all the titles
# db.get_table('movies').select(['title'])
db.get_table('movies').select(['title'], limit = 10)

[{'title': 'Toy Story (1995)'},
 {'title': 'Jumanji (1995)'},
 {'title': 'Grumpier Old Men (1995)'},
 {'title': 'Waiting to Exhale (1995)'},
 {'title': 'Father of the Bride Part II (1995)'},
 {'title': 'Heat (1995)'},
 {'title': 'Sabrina (1995)'},
 {'title': 'Tom and Huck (1995)'},
 {'title': 'Sudden Death (1995)'},
 {'title': 'GoldenEye (1995)'}]

# Query Planner

In [220]:
class QueryPlan:
	def __init__(self):
		self.steps = []

	def add_step(self, step):
		self.steps.append(step)

	def list_steps(self):
		return str(list(self.steps))

In [221]:
class QueryPlanner:
    def __init__(self, database):
        self.database = database

    def create_plan(self, ast):
        """
        Create a query execution plan based on the given AST.
        """
        plan = QueryPlan()

        # Example: Plan for a simple SELECT query
        if isinstance(ast, SelectStatement):
            # Step 1: Full Table Scan
            plan.add_step(('FullTableScan', ast.table_name))

            # Step 2: Filter (WHERE clause)
            if ast.where_clause:
                plan.add_step(('Filter', ast.where_clause))

            # TODO - Step 3: Joins
            # for join in ast.joins:
            #     plan.add_step(('Join', join.table_name, join.on_condition))

            # Additional steps like aggregations can be added here

        return plan

In [248]:
# test this sucker
planner = QueryPlanner(db)
plan = planner.create_plan(ast)  # Assuming 'ast' is an AST from the parser

In [249]:
print(plan.list_steps())

[('FullTableScan', 'movies')]


# Query Optimizer

In [224]:
class QueryOptimizer:
    def __init__(self, database):
        self.database = database

    def optimize(self, plan):
        """
        Optimize the given query plan.
        """
        optimized_plan = self._apply_predicate_pushdown(plan)
        optimized_plan = self._apply_join_reordering(optimized_plan)
        return optimized_plan

    def _apply_predicate_pushdown(self, plan):
        """
        Apply predicate pushdown optimization.
        """
        # This is a placeholder for predicate pushdown logic.
        # In practice, you would modify the plan to move filters closer to the data source.
        return plan

    def _apply_join_reordering(self, plan):
        """
        Apply join reordering based on a simple heuristic or cost model.
        """
        # This is a placeholder for join reordering logic.
        # In practice, you would reorder joins based on size, indexes, or other factors.
        return plan

In [225]:
# test our dummy optimizer
optimizer = QueryOptimizer(db)
optimized_plan = optimizer.optimize(plan)  # Assuming 'plan' is from the QueryPlanner

In [226]:
print(optimized_plan.list_steps())

[('FullTableScan', 'movies')]


# The Execution Engine

## Executable Operations

In [227]:
# basic class with a cost function - 
# because our database will natively provide cost of execution for every query...

class BaseOperator:
    def __init__(self):
        self.rows_processed = 0

    def get_cost(self):
        return {'rows_processed': self.rows_processed}

# Extend this base class in SelectOperator, JoinOperator, GroupByOperator etc.

In [228]:
class TableScanOperator(BaseOperator):
	def __init__(self, table): # Todo: provide a condition filter here to reduce scans -  condition=None)
		
		# self.db = db
		# self.table = self.db.get_table(table)
		self.table = table
		# self.condition = condition

	def execute(self):
		# table_scan = (row for row in self.table)
		table_scan = self.table.table_scan()
		self.rows_processed = len(table_scan)
		return table_scan

	def get_cost(self):
		return super().get_cost()


In [233]:
class SelectOperator(BaseOperator):
	def __init__(self, input_operator, select_columns):
		super().__init__()
		self.input_operator = input_operator
		self.select_columns = select_columns
		self.result_limit = 25

	def execute(self):
		result = []
		for row in self.input_operator.execute():
			self.rows_processed += 1
			if self.rows_processed <= self.result_limit:
				selected_row = {col: row[col] for col in self.select_columns}
				result.append(selected_row)
			else:
				break
		return result

	def get_cost(self):
		return super().get_cost()

## Execution Plan

In [234]:
def create_executable_plan(plan, database):
    """
    Translate the optimized query plan into executable operations.
    """
    executable_plan = []
    for step in plan.steps:
        if step[0] == 'FullTableScan':
            table_name = step[1]
            executable_plan.append(TableScanOperator(database.get_table(table_name)))

        elif step[0] == 'Select':
            columns = step[1]
            input_operator = executable_plan[-1]  # The input is the result of the previous step
            executable_plan.append(SelectOperator(input_operator, columns))

        # elif step[0] == 'Join':
        #     # Assuming step format is ('Join', right_table_name, join_condition)
        #     right_table = database.get_table(step[1])
        #     join_condition = step[2]
        #     left_operator = executable_plan[-1]
        #     right_operator = TableScanOperator(right_table)
        #     executable_plan.append(JoinOperator(left_operator, right_operator, join_condition))

        # Add cases for other types of steps (e.g., 'GroupBy', 'Filter')

    return executable_plan


## Executing Queries

In [244]:
def execute_query(executable_plan):
	"""
	Execute the query by running each operation in the executable plan.
	"""
	for operator in executable_plan:
		cost = {}
		result = operator.execute()
		cost[str(operator)] = operator.get_cost()

	# The result of the last operation is the result of the entire query
	return result, cost


In [247]:
# Example query
sql_query = "SELECT genres FROM movies WHERE genres = 'Comedy'"

# Parsing the query to AST
tokens = tokenize(sql_query)
parser = Parser(iter(tokens))
ast = parser.parse()

# Creating the query plan
planner = QueryPlanner(db)
plan = planner.create_plan(ast)

# Optimizing the query plan
optimizer = QueryOptimizer(db)
optimized_plan = optimizer.optimize(plan)

# Creating the executable plan
executable_plan = create_executable_plan(optimized_plan, db)

# Executing the query
query_result, query_costs = execute_query(executable_plan)

# Output the result
print(query_costs)


add  genres  to list of columns
Parse: parse: columns =  ['genres']
Parse: parse: table_name =  movies
Parse:parse: where_clause =  None
SelectStatement: this Select Statement has 
Columns:  ['genres'] 
Table:  movies 
WHERE CLAUSE:  None
{'<__main__.TableScanOperator object at 0x000001B291BE9670>': {'rows_processed': 9742}}


Wait, was it this simple?   
Yea!

# Next

Building a more feature rich data engine